# Ekstraksi Kata Kunci Berita Online
Dalam hal ini mencari dimana kata - kata kunci atau kata - kata paling penting dalam suatu kalimat pada dokument berita yang telah dicrawling sebelumnya pada website kompas.com

In [1]:
# sambung ke google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Import Library

In [2]:
!pip install sastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.5 MB/s eta 0:00:00


In [3]:
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import networkx as nx
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import pandas as pd
import numpy as np
import nltk
import re
import joblib

nltk.download('stopwords')
nltk.download('punkt')
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
# load data
data = pd.read_csv('/content/drive/MyDrive/ppw/ppw_implementasi/tugas ppw/dataset/berita_kompas.csv')
data

,Judul_Artikel,Tanggal Publish,Content_Artikel,Category
0,Jadwal Timnas Indonesia Malam Ini di Piala Dun...,"13/11/2023, 14:14 WIB",KOMPAS.com - Jadwal timnas Indonesia malam ini...,Sports
1,Eks Pelatih Garuda soal 2 Pemain Keturunan di ...,"13/11/2023, 13:20 WIB",KOMPAS.com - Keberadaan dua pemain keturunan d...,Sports
2,Cerita Jurnalis Mali di Piala Dunia U17: Dari ...,"13/11/2023, 12:15 WIB",KOMPAS.com - Dua jurnalis Mali hadir langsung ...,Sports
3,"Piala Dunia U17 2023, Jurnalis Ekuador Bicara ...","13/11/2023, 11:30 WIB","Laporan langsung jurnalis Kompas.com, Ahmad Zi...",Sports
4,5 MVP Turnamen Bold Battle of Legends 2023 Jal...,"13/11/2023, 11:03 WIB",KOMPAS.com - Rangkaian turnamen esports Bold B...,Sports
...,...,...,...,...
805,Penunggak Pajak Kendaraan Diumumkan di SPBU Be...,"08/11/2023, 06:22 WIB","JAKARTA, KOMPAS.com - Corporate Secretary Pert...",Otomotif
806,[POPULER OTOMOTIF] Intip Bodi Bus Langka Milik...,"08/11/2023, 06:02 WIB","JAKARTA, KOMPAS.com - Seiring perubahan zaman ...",Otomotif
807,"Subaru DriveFest di Batam, Tutup Perayaan 50 T...","07/11/2023, 20:21 WIB","JAKARTA, KOMPAS.com - Subaru Indonesia menggel...",Otomotif
808,"Ada Balap Resmi, Bamsoet Minta Pengguna Moge T...","07/11/2023, 20:11 WIB","JAKARTA, KOMPAS.com - Salah satu grup terbesar...",Otomotif


In [5]:
# Hitung jumlah kelas
kelas = data['Category'].value_counts()
kelas

Sports      270
Edukasi     270
Otomotif    270
Name: Category, dtype: int64

In [6]:
data.isnull().sum()

Judul_Artikel      0
Tanggal Publish    0
Content_Artikel    0
Category           0
dtype: int64

In [7]:
data.duplicated().sum()

0

### Clean data
Yaitu digunakan untuk menghapus karakter tertentu dimana fungsi cleaning digunakan untuk membersihkan teks dalam kolom ‘Content_Artikel’. Ini menghapus karakter-karakter yang tidak relevan seperti tanda baca dan mengubah teks menjadi daftar kata-kata.

In [8]:
def cleaning(text):
  text = re.sub(r'[^\w\s.?!,]','', text).strip().lower()
  return text
data['clean_content'] = data['Content_Artikel'].apply(cleaning)
data['clean_content']

0      kompas.com  jadwal timnas indonesia malam ini ...
1      kompas.com  keberadaan dua pemain keturunan di...
2      kompas.com  dua jurnalis mali hadir langsung k...
3      laporan langsung jurnalis kompas.com, ahmad zi...
4      kompas.com  rangkaian turnamen esports bold ba...
                             ...                        
805    jakarta, kompas.com  corporate secretary perta...
806    jakarta, kompas.com  seiring perubahan zaman d...
807    jakarta, kompas.com  subaru indonesia menggela...
808    jakarta, kompas.com  salah satu grup terbesar ...
809    bogor, kompas.com  mazda mx30 sudah tidak dila...
Name: clean_content, Length: 810, dtype: object

### Tokenizing
Fungsi tokenizer digunakan untuk melakukan tokenisasi dan juga proses case folding pada teks yang telah dibersihkan. Ini mengubah teks menjadi token-token kata.

In [9]:
def tokenizer(text):
  text = text.lower()
  return sent_tokenize(text)

data["tokenizing"] = data['clean_content'].apply(tokenizer)
data["tokenizing"]

0      [kompas.com  jadwal timnas indonesia malam ini...
1      [kompas.com  keberadaan dua pemain keturunan d...
2      [kompas.com  dua jurnalis mali hadir langsung ...
3      [laporan langsung jurnalis kompas.com, ahmad z...
4      [kompas.com  rangkaian turnamen esports bold b...
                             ...                        
805    [jakarta, kompas.com  corporate secretary pert...
806    [jakarta, kompas.com  seiring perubahan zaman ...
807    [jakarta, kompas.com  subaru indonesia menggel...
808    [jakarta, kompas.com  salah satu grup terbesar...
809    [bogor, kompas.com  mazda mx30 sudah tidak dil...
Name: tokenizing, Length: 810, dtype: object

### Ekstraksi Kata Kunci
Memulai ekstraksi kata kunci pada berita 1 yang telah dilakukan tokenizing

In [10]:
corpus = data["tokenizing"][0]
corpus

['kompas.com  jadwal timnas indonesia malam ini pada lanjutan babak penysiihan grup piala dunia u17 2023 bakal menyajikan laga melawan panama.',
 'hasil bagus akan membuat garuda muda menjaga kans lolos ke babak berikutnya.',
 'timnas u17 indonesia akan melanjutkan kiprah pada piala dunia u17 2023 dengan menghadapi panama pada matchday kedua grup a. laga indonesia vs panama tersebut bakal digelar di stadion gelora bung tomo, surabaya, pada senin 13112023 pukul 19.00 wib.',
 'baca juga indonesia tahan ekuador di piala dunia u17 laga berat, hasil kerja keras\xa0 pada laga perdana kontra ekuador, timnas u17 indonesia besutan bima sakti bermain imbang 11. garuda muda sebenarnya bisa unggul lebih dulu pada menit ke22 lewat gol arkhan kaka.',
 'namun, ekuador bisa membalas pada menit ke28 melalui gol allen obando.',
 'sementara itu, panama kalah 02 dari maroko pada pertandingan pertama grup a. dua gol maroko yang bersarang ke gawang panama dicetak oleh saifdine chlaghmo 16 dan ayman ennair 9

In [14]:
# Fungsi matriks co-occurence
from nltk.tokenize import word_tokenize
def build_co_occurrence_matrix(corpus,window_size):
    #build unique words
    unique_words=set()
    for text in corpus:
        for word in word_tokenize(text):
            unique_words.add(word)

    word_search_dict={word:np.zeros(shape=(len(unique_words))) for word in unique_words}
    word_list=list(word_search_dict.keys())
    for text in corpus:
        text_list=word_tokenize(text)
        for idx,word in enumerate(text_list):
            #pick word in the size range
            i=max(0,idx-window_size)
            j=min(len(text_list)-1,idx+window_size)
            search=[text_list[idx_] for idx_ in range(i,j+1)]
            search.remove(word)
            for neighbor in search:
                # get neighbor idx in word_search_dict
                nei_idx=word_list.index(neighbor)
                word_search_dict[word][nei_idx]+=1
    return word_search_dict

In [15]:
# Pemanggilan fungsi dengan data berita 1
coo_dict=build_co_occurrence_matrix(corpus,window_size=1)
coo_dict

{'perdana': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]),
 'menjaga': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 

In [16]:
# Ubah hasil ke dalam datafram
co_occurrence_matrix = pd.DataFrame(coo_dict,index=coo_dict.keys()).astype('int')
co_occurrence_matrix

,perdana,menjaga,19.00,pertama,matchday,13112023,bermain,maroko,jadwal,u17,...,kompas.com,kemenangan,welber,senin,juga,kans,pesan,905.,satu,ayman
perdana,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
menjaga,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
19.00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
pertama,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
matchday,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
kans,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
pesan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
905.,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
satu,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
# Menampilkan jumlah kemunculan kata
a = co_occurrence_matrix.stack()
a = a[a >= 1].rename_axis(('source', 'target')).reset_index(name='weight')
print(a)

      source      target  weight
0    perdana      kontra       1
1    perdana        laga       1
2    menjaga        muda       1
3    menjaga        kans       1
4      19.00         wib       1
..       ...         ...     ...
345     905.        baca       1
346     satu        poin       1
347     satu  mengoleksi       1
348    ayman         dan       1
349    ayman      ennair       1

[350 rows x 3 columns]


### Graph Co-occurence Matrics
Fungsi graph_co_occurrence ini bertujuan untuk membuat graf berdasarkan matriks co-occurrence dari teks yang diberikan

In [19]:
def graph_co_occurrence(x, threshold=0):
    vectorizer = CountVectorizer()
    tfidf_matrics = vectorizer.fit_transform(x)
    co_occurrence_matrix = tfidf_matrics.T.dot(tfidf_matrics).toarray()

    G = nx.Graph()

    # Menambahkan edge ke graf berdasarkan matriks co-occurrence
    for i in range(len(co_occurrence_matrix)):
      for j in range(i + 1, len(co_occurrence_matrix)):
        weight = co_occurrence_matrix[i, j]
        if weight > threshold:
          G.add_edge(vectorizer.get_feature_names_out()[i], vectorizer.get_feature_names_out()[j], weight=weight)
    return G

### Plot Graph Fuction
Fungsi plot_graph digunakan untuk menggambar grafik jaringan (network graph) berdasarkan graf yang diberikan (biasanya hasil dari fungsi graph_cosine_sim). Fungsi ini digunakan untuk visualisasi hubungan antara kalimat-kalimat dalam teks berdasarkan kesamaan kosinus.

In [20]:
def plot_graph(G, figsize=(35, 30), node_size=700, node_color='skyblue'):
  # Menggambar graf dengan canvas yang diperbesar
  pos = nx.spring_layout(G)  # Menentukan posisi simpul
  labels = nx.get_edge_attributes(G, 'weight')

  # Menentukan ukuran canvas
  plt.figure(figsize=(35, 30))

  # Menggambar graf dengan ukuran canvas yang diperbesar
  nx.draw(G, pos, with_labels=True, node_size=700, node_color='skyblue')
  nx.draw_networkx_edge_labels(G, pos, edge_labels=labels, font_color='red')
  plt.show()

### Peringkasan
Fungsi text_rank_summarization ini digunakan untuk melakukan ringkasan teks menggunakan algoritma TextRank.

In [21]:
def text_rank_summarization(x, k=4, index=1, threshold=0.11, show_scores=False):
    G = graph_co_occurrence(x, threshold)

    # Menghitung nilai dari PageRank (TextRank)
    scores = nx.pagerank(G)

    # Menyusun Kalimat berdasarkan nilai TextRank Scores tertinggi
    ranked_sentences = sorted(((sum(scores.get(word, 0) for word in kalimat.split()), kalimat) for kalimat in x), reverse=True)

    if show_scores:
      print(f"TextRank Scores Dokumen Ke - {index} : {scores}")
      # Menampilkan hasil peringkatan kalimat
      for skor, kalimat in ranked_sentences:
          print(f"Skor: {skor}, Kalimat: {kalimat}")

    summary_sentences = [s for _, s in ranked_sentences[:k]]

    return (' '.join(summary_sentences), G)

In [22]:
# Nilai Closeness
summary, G = text_rank_summarization(data["tokenizing"].iloc[:1][0], k = 3, show_scores=True, threshold=0)

TextRank Scores Dokumen Ke - 1 : {'00': 0.00580595350805628, '13112023': 0.00580595350805628, '19': 0.00580595350805628, '2023': 0.014782940205320583, 'akan': 0.007858099852257326, 'bakal': 0.00862054487034942, 'bung': 0.00580595350805628, 'dengan': 0.011960784863608338, 'di': 0.013221926846428315, 'digelar': 0.00580595350805628, 'dunia': 0.019932461668933147, 'gelora': 0.00580595350805628, 'grup': 0.020719192136945917, 'indonesia': 0.031494999886636246, 'kedua': 0.008055971915837873, 'kiprah': 0.00580595350805628, 'laga': 0.018693286529247895, 'matchday': 0.00580595350805628, 'melanjutkan': 0.00580595350805628, 'menghadapi': 0.00580595350805628, 'pada': 0.03554817470397126, 'panama': 0.030832604665062702, 'piala': 0.019932461668933147, 'pukul': 0.0058059535080562785, 'senin': 0.0058059535080562785, 'stadion': 0.0058059535080562785, 'surabaya': 0.0058059535080562785, 'tersebut': 0.0058059535080562785, 'timnas': 0.019932461668933154, 'tomo': 0.0058059535080562785, 'u17': 0.0351803006102

In [23]:
# Bentuk Graph
plot_graph(G)

Output hidden; open in https://colab.research.google.com to view.

In [24]:
# Hitung centrality pada berita 1 dengan closeness
closeness_centrality = nx.closeness_centrality(G)
# Ubah ke dalam dataframe
closeness_df = pd.DataFrame(list(closeness_centrality.items()), columns=['Kata', 'Closeness Centrality'])
closeness_df = closeness_df.sort_values(by='Closeness Centrality', ascending=False)
closeness_df

,Kata,Closeness Centrality
13,indonesia,0.916031
20,pada,0.916031
10,dunia,0.875912
30,u17,0.875912
22,piala,0.875912
...,...,...
108,poin,0.495868
107,peringkat,0.495868
106,mengoleksi,0.495868
105,berada,0.495868


In [25]:
# Hitung centrality pada berita 1 dengan PageRank
centrality = nx.pagerank(G)
# Ubah ke dalam dataframe
pagerank_df = pd.DataFrame(list(centrality.items()), columns=['Kata', 'Pagerank Centrality'])
pagerank_df = pagerank_df.sort_values(by='Pagerank Centrality', ascending=False)
pagerank_df

,Kata,Pagerank Centrality
20,pada,0.035548
30,u17,0.035180
13,indonesia,0.031495
21,panama,0.030833
12,grup,0.020719
...,...,...
114,menjaga,0.003190
113,lolos,0.003190
112,kans,0.003190
111,berikutnya,0.003190


In [26]:
# Mengurutkan peringkat kata-kata berdasarkan centrality
sorted_centrality = sorted(centrality.items(), key=lambda x: x[1], reverse=True)

# Buat dataframe dari hasil peringkat
df = pd.DataFrame(sorted_centrality, columns=['Kata', 'Centrality'])

# Tambahkan kolom peringkat
df['Peringkat'] = df['Centrality'].rank(ascending=False).astype(int)

df

,Kata,Centrality,Peringkat
0,pada,0.035548,1
1,u17,0.035180,2
2,indonesia,0.031495,3
3,panama,0.030833,4
4,grup,0.020719,5
...,...,...,...
116,bagus,0.003190,119
117,berikutnya,0.003190,119
118,kans,0.003190,119
119,lolos,0.003190,119


In [27]:
# Mengambil lima kata teratas
top_5_words = sorted_centrality[:5]

# Membuat DataFrame dari lima kata teratas
top_5_df = pd.DataFrame(top_5_words, columns=['Kata', 'Centrality'])
top_5_df

,Kata,Centrality
0,pada,0.035548
1,u17,0.035180
2,indonesia,0.031495
3,panama,0.030833
4,grup,0.020719
